### 코드 전체 흐름
1. 문서의 내용을 읽는다
2. 문서를 쪼갠다.
- 토큰수 초과로 답변을 생성하지 못할 수 있고
- 문서가 길면(Input이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 DB에 저장
4. 질문이 있을 때, 벡터 DB에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

### 1. 문서의 내용을 읽는다
- `LangChain` 을 쓸 때는 `Document Loader`를 사용했었다.
- `LangChain` 없이 문서의 내용을 읽을려면 `Python Docx` 를 사용한다.

In [ ]:
%pip install python-docx

In [ ]:
from docx import Document

document = Document('./tax.docx');
print(f'document == {document}')
full_text = ''

for index, paragraph in enumerate(document.paragraphs):
    print(f'paragraph == {paragraph.text}')
    full_text += f'{paragraph.text}\n'

In [ ]:
full_text

## 2. 문서를 쪼갠다.
- `LangChain` 을 쓸 때는 ``Text Spliter`` 를 사용했었다.
- `LangChain` 없이 문서의 내용을 읽을려면 OpenAI 에서 제공하는 `tiktoken` 을 사용한다.

In [ ]:
%pip install tiktoken

In [ ]:
import tiktoken

def split_text(input_text, chunk_size):
    encoder = tiktoken.encoding_for_model("gpt-4o")
    total_encoding = encoder.encode(input_text) # 인코딩 : 텍스트 -> 숫자 리스트
    total_token_count = len(total_encoding)
    text_list = []

    for i in range(0, total_token_count, chunk_size):
        chunk = total_encoding[i: i+chunk_size]
        decoded = encoder.decode(chunk)   # 디코딩 : 숫자 리스트 -> 텍스트
        text_list.append(decoded)

    return text_list



In [ ]:
chunk_list = split_text(full_text, 1500)

In [ ]:
chunk_list

## 3. 문서 임베딩
- `Chroma` 에서 제공하는 임베딩 기술을 사용한다.
- 기존에는 `langchain_chroma` 의 `Chroma` 를 사용했지만 이제는 그냥 `Chroma` 를 사용한다.

In [ ]:
%pip install chromadb

In [ ]:
import chromadb

chroma_client = chromadb.Client()

In [ ]:
collection_name = 'tax_collection'
tax_collection = chroma_client.create_collection(collection_name)

In [ ]:
import os
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_embedding = OpenAIEmbeddingFunction(api_key = openai_api_key,model_name='text-embedding-3-large')

In [ ]:
tax_collection = chroma_client.get_or_create_collection(collection_name, embedding_function=openai_embedding)

In [ ]:
id_list = []
for index in range(len(chunk_list)):
    id_list.append(f'{index}')

In [ ]:
tax_collection.add(documents=chunk_list, ids=id_list)

## 4. 유사도 검색

In [ ]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_doc = tax_collection.query(query_texts=query, n_results=3)

In [ ]:
retrieved_doc['documents'][0]

## 5. LLM 질의
- OpenAI 패키지가 필요함

In [ ]:
%pip install openai

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": f"당신은 한국의 소득세 전문가 입니다. 아래 내용을 참고해서 사용자의 질문에 답변해주세요 {retrieved_doc['documents'][0]}"},
        {"role": "user", "content": query},
    ]
)

In [ ]:
response.choices[0].message.content